In [1]:
import pyabf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob

import time
import datetime
from pathlib import Path

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


# Read in metadata from Shreejoy's github repo

In [2]:
# read final csv that has the ouput of metadata gathering process
nat_com_meta_url = 'https://raw.githubusercontent.com/stripathy/valiante_lab_abf_process/master/output_files/cell_final_raw_meta_df.csv'
cell_final_raw_meta_df = pd.read_csv(nat_com_meta_url)

In [3]:
cell_final_raw_meta_df.head()

,Unnamed: 0,cell_id,expt_date,layer_name,cell_type,recorder_name,full_path,abf_tag,abf_version,file_time,...,stim_amp_vec,stim_chan,stim_duration,stim_end_time,stim_gain,stim_name,stim_path,stim_sampling_rate,stim_start_time,valid_stim
0,0,13n05011.abf,2013-11-05,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,NaN,1.8.3.0,2013-11-05 17:06:16.448,...,[-400. -350. -300. ... 200. 250. 300.],0,0.6,0.7321,1000.0,sweepC,/Users/stripathy/Downloads/homeira_new_intrins...,10000,0.1322,True
1,1,13n21007.abf,2013-11-21,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,NaN,1.8.3.0,2013-11-21 14:30:46.356,...,[-400. -350. -300. ... 200. 250. 300.],0,0.6,0.7321,1000.0,sweepC,/Users/stripathy/Downloads/homeira_new_intrins...,10000,0.1322,True
2,2,13d02004.abf,2013-12-02,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,NaN,1.8.3.0,2013-12-02 15:47:42.278,...,[-400. -350. -300. ... 200. 250. 300.],0,0.6,0.7647,1000.0,sweepC,/Users/stripathy/Downloads/homeira_new_intrins...,10000,0.1648,True
3,3,13d02022.abf,2013-12-02,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,NaN,1.8.3.0,2013-12-02 17:09:07.433,...,[-400. -350. -300. ... 200. 250. 300.],0,0.6,0.7647,1000.0,sweepC,/Users/stripathy/Downloads/homeira_new_intrins...,10000,0.1648,True
4,4,13d02049.abf,2013-12-02,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,gain50 resting -57mv,1.8.3.0,2013-12-02 18:21:58.566,...,[-400. -350. -300. ... 200. 250. 300.],0,0.6,0.7647,1000.0,sweepC,/Users/stripathy/Downloads/homeira_new_intrins...,10000,0.1648,True


# find abf files on file path and extract abf file metadata

In [4]:
# The following provides a dataframe for all abf files found recursively within a directory
# it outputs the filename, path to file and date created

def summary_df(path, suffix='.abf'):
    all_files = []
    for i in path.rglob(f'*[!.DS_Store]{suffix}'): # searches for all files recursively excluding those named .DS_Store
        all_files.append((i.name,  i))

    columns = ["file_name", "path"]
    df = pd.DataFrame.from_records(all_files, columns=columns)
    df.path = df.path.astype('str')
    return df


nat_com_abf_dir = Path("./data/raw/recordings/Human tissue/nat comm data")
nat_com_abf_file_info = summary_df(nat_com_abf_dir)

In [5]:
%%time
def parse_timestamp(abf_path):
    try:
        abf = pyabf.ABF(abf_path)
        return datetime.datetime.strptime(abf.abfDateTimeString, "%Y-%m-%dT%H:%M:%S.%f")
    except UnicodeDecodeError:
        return None

def extract_abf_comments(abf_path):
    try:
        abf = pyabf.ABF(abf_path)
        return abf.tagComments
    except UnicodeDecodeError:
        return None

nat_com_abf_file_info['timestamp'] = nat_com_abf_file_info.path.apply(parse_timestamp)
nat_com_abf_file_info['abf_comments'] = nat_com_abf_file_info.path.apply(extract_abf_comments)

# add this column to merge with provided metadata
nat_com_abf_file_info['file_id'] = nat_com_abf_file_info.file_name.apply(lambda x: Path(x).stem) 

CPU times: user 2.68 s, sys: 972 ms, total: 3.65 s
Wall time: 3.9 s


In [6]:
nat_com_abf_file_info.tail()

,file_name,path,timestamp,abf_comments,file_id
131,2019_09_03_0036.abf,data/raw/recordings/Human tissue/nat comm data...,2019-09-03 15:05:10.703,"[C3, L2-RMP -65.8 mv]",2019_09_03_0036
132,19320025.abf,data/raw/recordings/Human tissue/nat comm data...,2019-03-20 18:20:39.921,"[C6, L2/3, RMP -66 mv,]",19320025
133,2015_11_09_0003.abf,data/raw/recordings/Human tissue/nat comm data...,2015-11-09 12:58:49.968,"[C#1, RMP -62.4 mV, L2/3]",2015_11_09_0003
134,2015_11_09_0017.abf,data/raw/recordings/Human tissue/nat comm data...,2015-11-09 13:40:54.390,"[C#2, RMP -61.9 mV, L2/3,, C#2, RMP -61.9 mV, ...",2015_11_09_0017
135,2016_03_01_0000.abf,data/raw/recordings/Human tissue/nat comm data...,2016-03-01 13:48:02.765,"[C#1, RMP -65.5,L2/3]",2016_03_01_0000


In [7]:
epileptogenic_ids = ['X2014.01.27', 'X2015.06.22', 'X2015.07.23']

In [8]:
nat_com_abf_file_info.file_id[nat_com_abf_file_info.file_id.isin(epileptogenic_ids)]

Series([], Name: file_id, dtype: object)

In [9]:
cell_final_raw_meta_df

,Unnamed: 0,cell_id,expt_date,layer_name,cell_type,recorder_name,full_path,abf_tag,abf_version,file_time,...,stim_amp_vec,stim_chan,stim_duration,stim_end_time,stim_gain,stim_name,stim_path,stim_sampling_rate,stim_start_time,valid_stim
0,0,13n05011.abf,2013-11-05,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,NaN,1.8.3.0,2013-11-05 17:06:16.448,...,[-400. -350. -300. ... 200. 250. 300.],0,0.6,0.73210,1000.0,sweepC,/Users/stripathy/Downloads/homeira_new_intrins...,10000,0.1322,True
1,1,13n21007.abf,2013-11-21,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,NaN,1.8.3.0,2013-11-21 14:30:46.356,...,[-400. -350. -300. ... 200. 250. 300.],0,0.6,0.73210,1000.0,sweepC,/Users/stripathy/Downloads/homeira_new_intrins...,10000,0.1322,True
2,2,13d02004.abf,2013-12-02,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,NaN,1.8.3.0,2013-12-02 15:47:42.278,...,[-400. -350. -300. ... 200. 250. 300.],0,0.6,0.76470,1000.0,sweepC,/Users/stripathy/Downloads/homeira_new_intrins...,10000,0.1648,True
3,3,13d02022.abf,2013-12-02,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,NaN,1.8.3.0,2013-12-02 17:09:07.433,...,[-400. -350. -300. ... 200. 250. 300.],0,0.6,0.76470,1000.0,sweepC,/Users/stripathy/Downloads/homeira_new_intrins...,10000,0.1648,True
4,4,13d02049.abf,2013-12-02,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,gain50 resting -57mv,1.8.3.0,2013-12-02 18:21:58.566,...,[-400. -350. -300. ... 200. 250. 300.],0,0.6,0.76470,1000.0,sweepC,/Users/stripathy/Downloads/homeira_new_intrins...,10000,0.1648,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,207,2020_01_27_0042.abf,2020-01-27,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,"C7 , L5 ,RMP -60.7 mv",2.6.0.0,2020-01-27 20:55:03.437,...,[-300. -290. -280. ... 460. 470. 480.],1,0.6,0.69205,1.0,sweepC,/Users/stripathy/Downloads/homeira_new_intrins...,20000,0.0921,True
208,210,2020_01_28_0029.abf,2020-01-28,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,NaN,2.6.0.0,2020-01-28 20:59:37.984,...,[-400. -350. -300. ... 300. 350. 400.],1,0.6,0.69205,1.0,sweepC,/Users/stripathy/Downloads/homeira_new_intrins...,20000,0.0921,True
209,208,2020_01_28_0008.abf,2020-01-28,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,NaN,2.6.0.0,2020-01-28 17:23:29.906,...,[-400. -350. -300. ... 300. 350. 400.],1,0.6,0.69205,1.0,sweepC,/Users/stripathy/Downloads/homeira_new_intrins...,20000,0.0921,True
210,209,2020_01_28_0017.abf,2020-01-28,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,"C4, L5, RMP -56.3 mV",2.6.0.0,2020-01-28 18:31:27.390,...,[-400. -350. -300. ... 300. 350. 400.],1,0.6,0.69205,1.0,sweepC,/Users/stripathy/Downloads/homeira_new_intrins...,20000,0.0921,True


# merge abf file metadata with old metadata sheet

In [10]:
merged_meta = cell_final_raw_meta_df.merge(nat_com_abf_file_info, left_on='cell_id', right_on='file_name' , how='inner')
merged_meta

,Unnamed: 0,cell_id,expt_date,layer_name,cell_type,recorder_name,full_path,abf_tag,abf_version,file_time,...,stim_name,stim_path,stim_sampling_rate,stim_start_time,valid_stim,file_name,path,timestamp,abf_comments,file_id
0,0,13n05011.abf,2013-11-05,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,NaN,1.8.3.0,2013-11-05 17:06:16.448,...,sweepC,/Users/stripathy/Downloads/homeira_new_intrins...,10000,0.1322,True,13n05011.abf,data/raw/recordings/Human tissue/nat comm data...,2013-11-05 17:06:16.448,[],13n05011
1,1,13n21007.abf,2013-11-21,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,NaN,1.8.3.0,2013-11-21 14:30:46.356,...,sweepC,/Users/stripathy/Downloads/homeira_new_intrins...,10000,0.1322,True,13n21007.abf,data/raw/recordings/Human tissue/nat comm data...,2013-11-21 14:30:46.356,[],13n21007
2,2,13d02004.abf,2013-12-02,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,NaN,1.8.3.0,2013-12-02 15:47:42.278,...,sweepC,/Users/stripathy/Downloads/homeira_new_intrins...,10000,0.1648,True,13d02004.abf,data/raw/recordings/Human tissue/nat comm data...,2013-12-02 15:47:42.278,[],13d02004
3,3,13d02022.abf,2013-12-02,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,NaN,1.8.3.0,2013-12-02 17:09:07.433,...,sweepC,/Users/stripathy/Downloads/homeira_new_intrins...,10000,0.1648,True,13d02022.abf,data/raw/recordings/Human tissue/nat comm data...,2013-12-02 17:09:07.433,[],13d02022
4,4,13d02049.abf,2013-12-02,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,gain50 resting -57mv,1.8.3.0,2013-12-02 18:21:58.566,...,sweepC,/Users/stripathy/Downloads/homeira_new_intrins...,10000,0.1648,True,13d02049.abf,data/raw/recordings/Human tissue/nat comm data...,2013-12-02 18:21:58.566,[gain50 resting -57mv],13d02049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,207,2020_01_27_0042.abf,2020-01-27,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,"C7 , L5 ,RMP -60.7 mv",2.6.0.0,2020-01-27 20:55:03.437,...,sweepC,/Users/stripathy/Downloads/homeira_new_intrins...,20000,0.0921,True,2020_01_27_0042.abf,data/raw/recordings/Human tissue/nat comm data...,2020-01-27 20:55:03.437,"[C7 , L5 ,RMP -60.7 mv]",2020_01_27_0042
126,210,2020_01_28_0029.abf,2020-01-28,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,NaN,2.6.0.0,2020-01-28 20:59:37.984,...,sweepC,/Users/stripathy/Downloads/homeira_new_intrins...,20000,0.0921,True,2020_01_28_0029.abf,data/raw/recordings/Human tissue/nat comm data...,2020-01-28 20:59:37.984,[],2020_01_28_0029
127,208,2020_01_28_0008.abf,2020-01-28,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,NaN,2.6.0.0,2020-01-28 17:23:29.906,...,sweepC,/Users/stripathy/Downloads/homeira_new_intrins...,20000,0.0921,True,2020_01_28_0008.abf,data/raw/recordings/Human tissue/nat comm data...,2020-01-28 17:23:29.906,[],2020_01_28_0008
128,209,2020_01_28_0017.abf,2020-01-28,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,"C4, L5, RMP -56.3 mV",2.6.0.0,2020-01-28 18:31:27.390,...,sweepC,/Users/stripathy/Downloads/homeira_new_intrins...,20000,0.0921,True,2020_01_28_0017.abf,data/raw/recordings/Human tissue/nat comm data...,2020-01-28 18:31:27.390,"[C4, L5, RMP -56.3 mV]",2020_01_28_0017


In [11]:
cleaned_meta = pd.read_csv('./data/processed/meta/human_meta_cleaned.csv')

In [12]:
merged_meta.iloc[0]

Unnamed: 0                                                            0
cell_id                                                    13n05011.abf
expt_date                                                    2013-11-05
layer_name                                                           L5
cell_type                                                           Pyr
recorder_name                                                   Homeira
full_path             /Users/stripathy/Downloads/homeira_new_intrins...
abf_tag                                                             NaN
abf_version                                                     1.8.3.0
file_time                                       2013-11-05 17:06:16.448
num_sweeps                                                           15
resp_chan                                                             0
resp_gain                                                          20.0
resp_offset                                                     

In [13]:
merged_meta = merged_meta.merge(cleaned_meta.loc[:, ['ABF File', 'subject_id', 'sex', 'age', 'seizure_duration_years', 
                                                     'diagnosis_Epilepsy', 'diagnosis_Tumor', 'resection_location', 'drugs', 'internal_soln', 
                                                     'external_soln']], left_on='file_id', right_on='ABF File').drop('ABF File', axis=1)

In [14]:
to_convert = merged_meta

In [15]:
to_convert.shape

(123, 42)

# set up directory for storing converted nwb files

In [16]:
output_dir = Path("data/processed/recordings/human/natcom")
output_dir.mkdir(exist_ok=True)

# perform abf to nwb file conversion using file converters

In [17]:
to_convert

,Unnamed: 0,cell_id,expt_date,layer_name,cell_type,recorder_name,full_path,abf_tag,abf_version,file_time,...,subject_id,sex,age,seizure_duration_years,diagnosis_Epilepsy,diagnosis_Tumor,resection_location,drugs,internal_soln,external_soln
0,0,13n05011.abf,2013-11-05,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,NaN,1.8.3.0,2013-11-05 17:06:16.448,...,X2013.11.05,M,57,45,True,False,Right-ATL,"LSC, CZP, CBZ",Potassium gluconate,aCSF
1,1,13n21007.abf,2013-11-21,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,NaN,1.8.3.0,2013-11-21 14:30:46.356,...,X2013.11.21,M,27,11,True,False,Right-ATL,"LSC, LRZ, CLB",Potassium gluconate,aCSF
2,2,13d02004.abf,2013-12-02,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,NaN,1.8.3.0,2013-12-02 15:47:42.278,...,X2013.12.02,F,39,11,False,True,Right-ATL,"LSC, LRZ, LEV",Potassium gluconate,aCSF
3,3,13d02022.abf,2013-12-02,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,NaN,1.8.3.0,2013-12-02 17:09:07.433,...,X2013.12.02,F,39,11,False,True,Right-ATL,"LSC, LRZ, LEV",Potassium gluconate,aCSF
4,4,13d02049.abf,2013-12-02,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,gain50 resting -57mv,1.8.3.0,2013-12-02 18:21:58.566,...,X2013.12.02,F,39,11,False,True,Right-ATL,"LSC, LRZ, LEV",Potassium gluconate,aCSF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,207,2020_01_27_0042.abf,2020-01-27,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,"C7 , L5 ,RMP -60.7 mv",2.6.0.0,2020-01-27 20:55:03.437,...,X2020.01.27,M,24,4,True,False,Left-ATL,"LEV, LTG",Potassium gluconate,aCSF
119,210,2020_01_28_0029.abf,2020-01-28,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,NaN,2.6.0.0,2020-01-28 20:59:37.984,...,X2020.01.28,M,33,6,True,False,Right-ATL,"LSc,PGB,PHN",Potassium gluconate,aCSF
120,208,2020_01_28_0008.abf,2020-01-28,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,NaN,2.6.0.0,2020-01-28 17:23:29.906,...,X2020.01.28,M,33,6,True,False,Right-ATL,"LSc,PGB,PHN",Potassium gluconate,aCSF
121,209,2020_01_28_0017.abf,2020-01-28,L5,Pyr,Homeira,/Users/stripathy/Downloads/homeira_new_intrins...,"C4, L5, RMP -56.3 mV",2.6.0.0,2020-01-28 18:31:27.390,...,X2020.01.28,M,33,6,True,False,Right-ATL,"LSc,PGB,PHN",Potassium gluconate,aCSF


In [18]:
to_convert.recorder_name.value_counts()

Homeira    123
Name: recorder_name, dtype: int64

In [19]:
from x_to_nwb import ABF2Converter
from x_to_nwb import ABF1Converter

In [20]:
to_convert.iloc[0]

Unnamed: 0                                                                0
cell_id                                                        13n05011.abf
expt_date                                                        2013-11-05
layer_name                                                               L5
cell_type                                                               Pyr
recorder_name                                                       Homeira
full_path                 /Users/stripathy/Downloads/homeira_new_intrins...
abf_tag                                                                 NaN
abf_version                                                         1.8.3.0
file_time                                           2013-11-05 17:06:16.448
num_sweeps                                                               15
resp_chan                                                                 0
resp_gain                                                              20.0
resp_offset 

In [21]:
%%time
errors = []

for i, file_info in to_convert.iterrows():
    print(file_info)
    try:
        abf = pyabf.ABF(file_info.path) # abf file to be converted
        file_stem = file_info.cell_id.rstrip('.abf') # file stem with no file type
        
        # figure out if stimulus is on dac or adc channels
        if file_info.stim_name == 'sweepC': 
            stim_chan = abf.dacNames[file_info.stim_chan] 
        else: #if stim_name is sweepY
            stim_chan = abf.adcNames[file_info.stim_chan] 
        
        resp_chan = abf.adcNames[file_info.resp_chan] # response seems always to be in adc channels
        
        # this should be populated with metadata from homeira!
        meta = {
            'Subject': {
                'age': f'P{file_info.age}Y',
                'description': f'{file_info.abf_tag}',
                'species': 'Homo sapiens',
                'subject_id': f'{file_info["subject_id"]}'
            }
        }

        
        output_path = output_dir / f'{file_stem}.nwb'
        print(output_path)
        
        if file_info.abf_version == '2.6.0.0': # if abf v2 file
            # i didn't use gain here because none of my abf v2 files needed a gain - feel free to add one though
            nwb = ABF2Converter.ABF2Converter(file_info.path, 
                                              str(output_path), 
                                              includeChannelList=[resp_chan], 
                                              stimulus_name=stim_chan, 
                                              metadata=meta)
        
        else: # if abf 1 file
            nwb = ABF1Converter.ABF1Converter(inputPath = file_info.path, 
                                                    outputFilePath = str(output_path), 
                                                    acquisitionChannelName = resp_chan, 
                                                    stimulusChannelName = stim_chan,
                                                    metadata=meta, 
                                                    responseGain = file_info.resp_gain, # gain on response channel
                                                    stimulusGain = file_info.stim_gain, # gain on stimulus channel
                                                    responseOffset = file_info.resp_offset, # voltage offset on resp channel
                                                    )
            nwb.convert()
        print(nwb)
        print("NWB Conversion complete for " + file_stem +'.nwb\n')
        
    except (ValueError, IndexError, TypeError, KeyError, UnicodeDecodeError) as e:
        print('**'*50)
        print(f'Error: {e} \n {file_info.path} \n')
        errors.append((e, file_info.file_id, file_info.path))

Unnamed: 0                                                                0
cell_id                                                        13n05011.abf
expt_date                                                        2013-11-05
layer_name                                                               L5
cell_type                                                               Pyr
recorder_name                                                       Homeira
full_path                 /Users/stripathy/Downloads/homeira_new_intrins...
abf_tag                                                                 NaN
abf_version                                                         1.8.3.0
file_time                                           2013-11-05 17:06:16.448
num_sweeps                                                               15
resp_chan                                                                 0
resp_gain                                                              20.0
resp_offset 

/Users/derek_howard/miniconda3/envs/nwbephystest/lib/python3.8/site-packages/pynwb/file.py:753: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")


Successfully converted to data/processed/recordings/human/natcom/13n05011.nwb.
NWB Conversion complete for 13n05011.nwb

Unnamed: 0                                                                1
cell_id                                                        13n21007.abf
expt_date                                                        2013-11-21
layer_name                                                               L5
cell_type                                                               Pyr
recorder_name                                                       Homeira
full_path                 /Users/stripathy/Downloads/homeira_new_intrins...
abf_tag                                                                 NaN
abf_version                                                         1.8.3.0
file_time                                           2013-11-21 14:30:46.356
num_sweeps                                                               15
resp_chan                                  

Successfully converted to data/processed/recordings/human/natcom/13d02022.nwb.
NWB Conversion complete for 13d02022.nwb

Unnamed: 0                                                                4
cell_id                                                        13d02049.abf
expt_date                                                        2013-12-02
layer_name                                                               L5
cell_type                                                               Pyr
recorder_name                                                       Homeira
full_path                 /Users/stripathy/Downloads/homeira_new_intrins...
abf_tag                                                gain50 resting -57mv
abf_version                                                         1.8.3.0
file_time                                           2013-12-02 18:21:58.566
num_sweeps                                                               15
resp_chan                                  

Successfully converted to data/processed/recordings/human/natcom/13d03007.nwb.
NWB Conversion complete for 13d03007.nwb

Unnamed: 0                                                                7
cell_id                                                        13d03008.abf
expt_date                                                        2013-12-03
layer_name                                                               L5
cell_type                                                               Pyr
recorder_name                                                       Homeira
full_path                 /Users/stripathy/Downloads/homeira_new_intrins...
abf_tag                                                                 NaN
abf_version                                                         1.8.3.0
file_time                                           2013-12-03 14:50:32.511
num_sweeps                                                               15
resp_chan                                  

...

NWB Conversion complete for 2020_01_28_0029.nwb

Unnamed: 0                                                              208
cell_id                                                 2020_01_28_0008.abf
expt_date                                                        2020-01-28
layer_name                                                               L5
cell_type                                                               Pyr
recorder_name                                                       Homeira
full_path                 /Users/stripathy/Downloads/homeira_new_intrins...
abf_tag                                                                 NaN
abf_version                                                         2.6.0.0
file_time                                           2020-01-28 17:23:29.906
num_sweeps                                                               17
resp_chan                                                                 0
resp_gain                              

/Users/derek_howard/triplab/x-to-nwb/src/x_to_nwb/ABF2Converter.py:188: UserWarning: Could not find the JSON file data/raw/recordings/Human tissue/nat comm data/Total5Homeira-Last-Version - with 2020 Data/2020_01_28_0017.json with settings.
  warnings.warn(f"Could not find the JSON file {settings} with settings.")


NWB Conversion complete for 2020_01_28_0008.nwb

Unnamed: 0                                                              209
cell_id                                                 2020_01_28_0017.abf
expt_date                                                        2020-01-28
layer_name                                                               L5
cell_type                                                               Pyr
recorder_name                                                       Homeira
full_path                 /Users/stripathy/Downloads/homeira_new_intrins...
abf_tag                                                C4, L5, RMP -56.3 mV
abf_version                                                         2.6.0.0
file_time                                           2020-01-28 18:31:27.390
num_sweeps                                                               17
resp_chan                                                                 0
resp_gain                              

/Users/derek_howard/triplab/x-to-nwb/src/x_to_nwb/ABF2Converter.py:188: UserWarning: Could not find the JSON file data/raw/recordings/Human tissue/nat comm data/Total5Homeira-Last-Version - with 2020 Data/2020_03_02_0023.json with settings.
  warnings.warn(f"Could not find the JSON file {settings} with settings.")


NWB Conversion complete for 2020_01_28_0017.nwb

Unnamed: 0                                                              211
cell_id                                                 2020_03_02_0023.abf
expt_date                                                        2020-03-02
layer_name                                                               L5
cell_type                                                               Pyr
recorder_name                                                       Homeira
full_path                 /Users/stripathy/Downloads/homeira_new_intrins...
abf_tag                                                C3-, RMp -68, 93, L5
abf_version                                                         2.6.0.0
file_time                                           2020-03-02 16:36:35.609
num_sweeps                                                               18
resp_chan                                                                 0
resp_gain                              

# perform dandi validation for all files in recordings folder

In [22]:
!dandi validate "data/processed/recordings/human/natcom"

2022-01-26 11:24:44,470 [ WARNING] A newer version (0.35.0) of dandi/dandi-cli is available. You are using 0.34.1
2022-01-26 11:24:44,749 [    INFO] NumExpr defaulting to 8 threads.
2022-01-26 11:24:47,700 [    INFO] data/processed/recordings/human/natcom/15o08032.nwb: ok
2022-01-26 11:24:49,768 [    INFO] data/processed/recordings/human/natcom/2016_03_03_0103.nwb: ok
2022-01-26 11:24:51,682 [    INFO] data/processed/recordings/human/natcom/2016_02_29_0065.nwb: ok
2022-01-26 11:24:53,556 [    INFO] data/processed/recordings/human/natcom/19320017.nwb: ok
2022-01-26 11:24:55,410 [    INFO] data/processed/recordings/human/natcom/19320001.nwb: ok
2022-01-26 11:24:57,211 [    INFO] data/processed/recordings/human/natcom/19128036.nwb: ok
2022-01-26 11:24:59,103 [    INFO] data/processed/recordings/human/natcom/2016_03_03_0100.nwb: ok
2022-01-26 11:25:01,035 [    INFO] data/processed/recordings/human/natcom/20140127_600_1_0118.nwb: ok
2022-01-26 11:25:02,905 [    INFO] data/processed/recordin

2022-01-26 11:27:40,923 [    INFO] data/processed/recordings/human/natcom/15o08007.nwb: ok
2022-01-26 11:27:43,690 [    INFO] data/processed/recordings/human/natcom/19129069.nwb: ok
2022-01-26 11:27:46,450 [    INFO] data/processed/recordings/human/natcom/19129057.nwb: ok
2022-01-26 11:27:48,909 [    INFO] data/processed/recordings/human/natcom/15o08038.nwb: ok
2022-01-26 11:27:51,126 [    INFO] data/processed/recordings/human/natcom/13d02049.nwb: ok
2022-01-26 11:27:53,300 [    INFO] data/processed/recordings/human/natcom/19128003.nwb: ok
2022-01-26 11:27:55,674 [    INFO] data/processed/recordings/human/natcom/20131211_600_1_0081.nwb: ok
2022-01-26 11:27:59,946 [    INFO] data/processed/recordings/human/natcom/2020_01_27_0042.nwb: ok
2022-01-26 11:28:02,271 [    INFO] data/processed/recordings/human/natcom/20131211_600_1_0042.nwb: ok
2022-01-26 11:28:04,716 [    INFO] data/processed/recordings/human/natcom/15o08011.nwb: ok
2022-01-26 11:28:07,116 [    INFO] data/processed/recordings/